In [129]:
# Load Libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from textblob import TextBlob
%matplotlib inline  

In [113]:
data=pd.read_csv("narendramodi_tweets.csv")
data.head(10)

,id,created_at,text,retweet_count,favorite_count
0,NaN,NaN,NaN,NaN,NaN
1,9.799140e+17,31-03-2018 02:49,b'\xe0\xa4\x86\xe0\xa4\xaa \xe0\xa4\xb8\xe0\xa...,7317.0,36351.0
2,NaN,NaN,NaN,NaN,NaN
3,9.797680e+17,30-03-2018 17:09,b'Appealed to my young friends to work on inno...,1814.0,8186.0
4,NaN,NaN,NaN,NaN,NaN
5,9.797670e+17,30-03-2018 17:08,"b'In the 21st century, IPPP will be a strong f...",2481.0,12015.0
6,NaN,NaN,NaN,NaN,NaN
7,9.797670e+17,30-03-2018 17:05,"b'Thanks to technology, today\xe2\x80\x99s Hac...",2709.0,15661.0
8,NaN,NaN,NaN,NaN,NaN
9,9.797660e+17,30-03-2018 17:03,b'Programmes like the Smart India Hackathon ma...,1682.0,7061.0


In [114]:
data_clean=data.dropna()
data_clean.head(10)

,id,created_at,text,retweet_count,favorite_count
1,9.799140e+17,31-03-2018 02:49,b'\xe0\xa4\x86\xe0\xa4\xaa \xe0\xa4\xb8\xe0\xa...,7317.0,36351.0
3,9.797680e+17,30-03-2018 17:09,b'Appealed to my young friends to work on inno...,1814.0,8186.0
5,9.797670e+17,30-03-2018 17:08,"b'In the 21st century, IPPP will be a strong f...",2481.0,12015.0
7,9.797670e+17,30-03-2018 17:05,"b'Thanks to technology, today\xe2\x80\x99s Hac...",2709.0,15661.0
9,9.797660e+17,30-03-2018 17:03,b'Programmes like the Smart India Hackathon ma...,1682.0,7061.0
11,9.797150e+17,30-03-2018 13:38,b'RT @PMOIndia: PM @narendramodi will be joini...,1572.0,0.0
13,9.796750e+17,30-03-2018 11:00,"b'Kutch, which witnessed massive devastation a...",3143.0,16212.0
15,9.796750e+17,30-03-2018 10:59,b'Happy to have addressed the Silver Jubilee C...,1723.0,8531.0
17,9.795520e+17,30-03-2018 02:54,b'On Good Friday we recall the courage and com...,4790.0,26390.0
19,9.795510e+17,30-03-2018 02:49,b'\xe0\xa4\xb5\xe0\xa5\x80\xe0\xa4\xb0-\xe0\xa...,4129.0,19615.0


In [115]:
stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [118]:
# clean description
data_clean['text'] = data_clean['text'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [119]:
data_clean.head(10)

,id,created_at,text,retweet_count,favorite_count
1,9.799140e+17,31-03-2018 02:49,bxe0xa4x86xe0xa4xaa xe0xa4xb8xe0xa4xadxe0xa5x8...,7317.0,36351.0
3,9.797680e+17,30-03-2018 17:09,bappeal young friend work innov transform live...,1814.0,8186.0
5,9.797670e+17,30-03-2018 17:08,bin 21st centuri ippp strong forc ippp mean in...,2481.0,12015.0
7,9.797670e+17,30-03-2018 17:05,bthank technolog todayxe2x80x99 hackathon gave...,2709.0,15661.0
9,9.797660e+17,30-03-2018 17:03,bprogramm like smart india hackathon make extr...,1682.0,7061.0
11,9.797150e+17,30-03-2018 13:38,brt pmoindia pm narendramodi join grand final ...,1572.0,0.0
13,9.796750e+17,30-03-2018 11:00,bkutch wit massiv devast 2001 quak prosper dis...,3143.0,16212.0
15,9.796750e+17,30-03-2018 10:59,bhappi address silver jubile celebr shree cutc...,1723.0,8531.0
17,9.795520e+17,30-03-2018 02:54,bon good friday recal courag compass lord chri...,4790.0,26390.0
19,9.795510e+17,30-03-2018 02:49,bxe0xa4xb5xe0xa5x80xe0xa4xb0xe0xa4xb5xe0xa5x80...,4129.0,19615.0


In [120]:
text=pd.DataFrame(data_clean['text'])
text.head(10)

,text
1,bxe0xa4x86xe0xa4xaa xe0xa4xb8xe0xa4xadxe0xa5x8...
3,bappeal young friend work innov transform live...
5,bin 21st centuri ippp strong forc ippp mean in...
7,bthank technolog todayxe2x80x99 hackathon gave...
9,bprogramm like smart india hackathon make extr...
11,brt pmoindia pm narendramodi join grand final ...
13,bkutch wit massiv devast 2001 quak prosper dis...
15,bhappi address silver jubile celebr shree cutc...
17,bon good friday recal courag compass lord chri...
19,bxe0xa4xb5xe0xa5x80xe0xa4xb0xe0xa4xb5xe0xa5x80...


In [121]:
def removeImage(text):
    words=text.split(" ")
    words=[word for word in words if(word!='b' and word[0]!='x' and word[0:2]!='bx' and word[0:2]!='rt')]
    return " ".join(words)

In [122]:
text['text']=text['text'].map(lambda x: removeImage(x))
text['text'].head(10)

1     everyon auspici occas hanuman jayanti httpstco...
3     bappeal young friend work innov transform live...
5     bin 21st centuri ippp strong forc ippp mean in...
7     bthank technolog todayxe2x80x99 hackathon gave...
9     bprogramm like smart india hackathon make extr...
11    brt pmoindia pm narendramodi join grand final ...
13    bkutch wit massiv devast 2001 quak prosper dis...
15    bhappi address silver jubile celebr shree cutc...
17    bon good friday recal courag compass lord chri...
19                                   httpstcow0gi2abvio
Name: text, dtype: object

In [138]:
def get_tweet_sentiment(tweet):
    #Utility function to classify sentiment of passed tweet
    #using textblob's sentiment method
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweet)
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [139]:
text['sentiment'] = text['text'].map(lambda x: get_tweet_sentiment(x))
text.head(10)

,text,sentiment
1,everyon auspici occas hanuman jayanti httpstco...,neutral
3,bappeal young friend work innov transform live...,positive
5,bin 21st centuri ippp strong forc ippp mean in...,positive
7,bthank technolog todayxe2x80x99 hackathon gave...,neutral
9,bprogramm like smart india hackathon make extr...,positive
11,brt pmoindia pm narendramodi join grand final ...,positive
13,bkutch wit massiv devast 2001 quak prosper dis...,neutral
15,bhappi address silver jubile celebr shree cutc...,neutral
17,bon good friday recal courag compass lord chri...,positive
19,httpstcow0gi2abvio,neutral


In [141]:
text[text['sentiment']=='negative']

,text,sentiment
87,bin less hundr day world mark 4thyogaday urg t...,negative
149,bwork steadfastli toward qualiti afford health...,negative
225,bwe attach topmost prioriti aspir poor neomidd...,negative
277,bthe willpow affect tb extrem inspir courag mo...,negative
307,brt defenceminindia 4 commando equip gp amp sa...,negative
379,bbirthday greet punjab cm captamarind ji pray ...,negative
431,brt himbhu131 mother inspir believ atmost fait...,negative
441,bcongratul mr neiphiu rio mr patton other took...,negative
463,bbirthday greet rajasthan cm vasundharabjp ji ...,negative
475,bwe never shi away take tough decis larger int...,negative


In [142]:
text[text['sentiment']=='positive']

,text,sentiment
3,bappeal young friend work innov transform live...,positive
5,bin 21st centuri ippp strong forc ippp mean in...,positive
9,bprogramm like smart india hackathon make extr...,positive
11,brt pmoindia pm narendramodi join grand final ...,positive
17,bon good friday recal courag compass lord chri...,positive
21,bat 8 pm tomorrow join grand final smart india...,positive
25,bgsat6a commun satellit provid new possibl mob...,positive
27,bcongratul isro stakehold success launch gslvf...,positive
31,bgreet mahavir jayanti nnbhagwan mahavirxe2x80...,positive
35,bwarmest congratul good wish excel u win myint...,positive
